<h3>Data Scraping</h3>

In [1]:
#import basic libraries
import pandas as pd
import re
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
from scrapy.pipelines.images import ImagesPipeline
from scrapy.exceptions import DropItem

class ImageItem(scrapy.Item):

    # for downloading images in ImagePipeline
    image_urls = scrapy.Field()
    
    #link to specific image
    image_links = scrapy.Field()
    
    #image attributes
    titles = scrapy.Field()
    date_posted = scrapy.Field()
    hashtags = scrapy.Field()
    
    #stats
    views = scrapy.Field()
    faves = scrapy.Field()
    comments = scrapy.Field()
    downloads = scrapy.Field()
    
    #artist details
    artists = scrapy.Field()
    artist_urls = scrapy.Field()
    artist_deviations = scrapy.Field()
    artist_comments = scrapy.Field()
    artist_page_views = scrapy.Field()
    artist_scraps = scrapy.Field()
    artist_watchers = scrapy.Field()
    artist_critiques = scrapy.Field()
    artist_forum_posts = scrapy.Field()
    artist_faves = scrapy.Field()
    artist_asl = scrapy.Field() #age, sex, location
    artist_dob = scrapy.Field() #date of birth
    account_age = scrapy.Field() #how old the account is
    membership = scrapy.Field() #artist current membership status
    
    # to be filled in by ImagePipeline
    image_paths = scrapy.Field()
    images = scrapy.Field()
    
class MyImagesPipeline(ImagesPipeline):

    def get_media_requests(self, item, info):
        for image_url in item['image_urls']:
            yield scrapy.Request(image_url)

    def item_completed(self, results, item, info):
        image_paths = [x['path'] for ok, x in results if ok]
        if not image_paths:
            raise DropItem("Item contains no images")
        item['image_paths'] = image_paths
        return item

In [3]:
class ImageSpider(scrapy.Spider):
    
    name = 'images'
    
    start_urls = ['https://www.deviantart.com/popular-all-time/?q=sherlock&offset=0']
    
    
    #initialize offset at 0
    offset = 0
    #set offset limit to control the amount of images downloaded
    offset_limit = 8000
    #count items
    old_items = 0
    new_items = 0
    

    custom_settings = {
        'LOG_LEVEL': logging.INFO,
        'ITEM_PIPELINES': {'__main__.MyImagesPipeline': 1}, #enable image download
        'IMAGES_STORE': 'DA-images', #store images in DA-images folder
        'FEED_FORMAT':'json',                                
        'FEED_URI': 'image-data.json', #store image data in image-data.json
        'DOWNLOAD_FAIL_ON_DATALOSS': False, #if image download fails (due to various issues), don't send error message, just flag it.
        'DOWNLOAD_DELAY': 0.25 #250 ms download delay, with inbuilt scrapy randomization
    }

    def parse(self, response):
        
        #get page body
        page = response.css('div.page-results span.thumb')
        
        for img in page:
            
            #thumbnail link. If there isn't a thumbnail, then post is not an image and should be skipped
            thumbnail = img.css('::attr(data-super-img)').get()
            
            #img_link contains the full link that leads to the individual image post
            img_link = img.css('::attr(href)').get()
            
            #if there is a thumbnail, aka the post is an image, follow url to parse image for details
            if thumbnail: 
                yield scrapy.Request(img_link, callback = self.parse_image)
                self.new_items += 1
                if (self.new_items - self.old_items) == 100:
                    self.old_items = self.new_items
                    print(f"Downloaded {self.new_items} images...")
                
        #go to next page
        #while self.offset < self.offset_limit:
            #self.offset += 24 #DA's natural offset scroll is set at increments of 24
            #next_page = f'https://www.deviantart.com/popular-all-time/?q=sherlock&offset={self.offset}'
            #yield scrapy.Request(next_page, callback=self.parse)
            
            
    def parse_image(self, response):
        
        #initialize image item
        image = ImageItem()
        
        #get image url (for downloading via ImagePipeline)
        image["image_urls"] = [response.css('div.dev-view-deviation img ::attr(src)').get()]
        #get other image info
        image["image_links"] = response.url
        image['titles'] = response.xpath("//a[@class='title']/text()").extract()[0]
        image['date_posted'] = response.xpath("//div[@class='dev-right-bar-content dev-metainfo-content dev-metainfo-details']/dl/dd/span/text()").extract()[0]
        
        #check whether image has hashtags (some don't)
        hashtag = response.xpath("//div[@class='dev-about-tags-cc dev-about-breadcrumb']/a/text()").extract()
        if hashtag: image['hashtags'] = hashtag
        
        #get image stats
        stats =  response.xpath("//div[@class='dev-right-bar-content dev-metainfo-content dev-metainfo-stats']/dl/dd/text()").extract()
        
        #check that stats list only contains numbers (sometimes irregular data falls in)
        stats = [re.sub("\D", "", s) for s in stats]
        
        #remove any None types from list
        stats = list(filter(None, stats))
        
        #the responses are ordered in: views, faves, comments, downloads
        #sometimes comments are disabled, sometimes downloads are disabled
        headers = ['views','faves','comments','downloads']
        
        #if comments/downloads are disabled, they will not be looped over for a given image
        for i in range(len(stats)):
            image[headers[i]] = stats[i]
            
        #get artist info
        artist_name = response.xpath("//small[@class='author']/span/a/text()").extract()
        
        if artist_name: 
            image['artists'] = response.xpath("//small[@class='author']/span/a/text()").extract()[-1]
            image['artist_urls'] = response.xpath("//small[@class='author']/span/a/@href").extract()[-1]     
            request = scrapy.Request(image['artist_urls'], callback=self.parse_artist, meta={'image':image})
            yield request
        else: #if no artist name (sometimes artists are banned), just yield the image
            image['artists'] = 'Banned'
            yield image
        
    def parse_artist(self, response):
        
        #get image item for the higher-level parser
        image = response.meta['image']
        
        #get artist account stats
        artist_stats = response.xpath("//div[@id='super-secret-stats']/div/div/div/strong/text()").extract()
        
        headers = ['artist_deviations','artist_comments','artist_page_views','artist_scraps','artist_watchers','artist_critiques','artist_forum_posts','artist_faves']
        
        for i in range(len(artist_stats)):
            image[headers[i]] = artist_stats[i]
        
        #get account age and membership details
        age_membership = response.xpath("//a[@href='#super-secret-activity']/div/text()").extract()
        
        #sometimes the age wrapped up in a span
        if len(age_membership) == 0: 
            age = response.xpath("//a[@href='#super-secret-activity']/span/div/text()").extract()[0]
            age_membership.append(age)        
        image['account_age'] = age_membership[0]

        
        #get artist personal details
        artist_details = response.xpath("//div[@id='super-secret-why']/div/div/div/dl/dd/text()").extract()
        details = ['artist_asl','artist_dob'] #some artists do not share their dobs
        for i in range(len(artist_details)):
            image[details[i]] = artist_details[i]
        
        return image
            
process = CrawlerProcess()
process.crawl(ImageSpider)
process.start()

2019-04-17 19:49:19 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-04-17 19:49:19 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.5.0, Python 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Darwin-18.2.0-x86_64-i386-64bit
2019-04-17 19:49:19 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': 0.25, 'DOWNLOAD_FAIL_ON_DATALOSS': False, 'FEED_FORMAT': 'json', 'FEED_URI': 'image-data.json', 'LOG_LEVEL': 20}
2019-04-17 19:49:19 [scrapy.extensions.telnet] INFO: Telnet Password: 7a1c3d2284e0984b
2019-04-17 19:49:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.exten

In [7]:
df = pd.read_json('image-data.json')
df

,account_age,artist_asl,artist_comments,artist_critiques,artist_deviations,artist_dob,artist_faves,artist_forum_posts,artist_page_views,artist_scraps,...,comments,date_posted,downloads,faves,hashtags,image_links,image_paths,image_urls,titles,views
0,Deviant for 5 Years,Female/France,468,0,183,March 5,"1,830",0,"10,611",0,...,22,"January 17, 2014",91.0,142,NaN,https://www.deviantart.com/get-sherlock/art/Sh...,[full/c11741fda71c47368b3ecc42de439de62c48cf30...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,1597
1,Deviant for 6 Years,People's Republic of China,97,0,66,June 18,"1,133",0,"7,418",0,...,20,"August 17, 2012",20.0,258,NaN,https://www.deviantart.com/jaslerb/art/sherloc...,[full/39524abdbf4128e28696adc9c450d6d59887d012...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,sherlock,3389
2,Deviant for 6 Years,Female/Japan,23,0,28,September 9,53,0,"9,900",0,...,102,"August 11, 2012",225.0,1961,NaN,https://www.deviantart.com/donperico/art/SHERL...,[full/0f13dd8004eb7f0ce2b45c826f4e76df5978ac13...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,SHERLOCK,21161
3,Deviant for 8 Years,Female/Russia,169,0,36,March 5,216,0,"11,368",2,...,45,"December 30, 2011",186.0,1135,NaN,https://www.deviantart.com/masterhalfling/art/...,[full/09fbbe05751a1b08cf03445e0821ca44dd5b547b...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,14150
4,Deviant for 7 Years,Unknown,56,0,42,NaN,73,0,"15,524",0,...,42,"January 30, 2012",283.0,1560,NaN,https://www.deviantart.com/unisha/art/Sherlock...,[full/1e8b904f0bb82d4f5dfca301704d95e6cab4273d...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,20889
5,Deviant for 7 Years,Female/South Korea,3,0,5,NaN,6,0,"15,726",0,...,168,"September 27, 2011",550.0,3848,NaN,https://www.deviantart.com/dotdodoe/art/sherlo...,[full/cf0367dd32b78ecd92d0e8906edb18e8fda32233...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,sherlock,42717
6,Deviant for 8 Years,Unknown,759,0,62,March 22,482,0,"89,325",0,...,322,"October 27, 2010",803.0,5494,NaN,https://www.deviantart.com/madtenka/art/sherlo...,[full/23fa7bc5787bbfc6ef45b1d916d9a22cf2dd5522...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,sherlock,86962
7,Deviant for 11 Years,Female/Russia,339,0,127,February 27,147,0,"49,622",0,...,88,"September 7, 2010",193.0,1702,[cumberbatchsherlock],https://www.deviantart.com/umino-aka-morskaya/...,[full/240c5af33c8a67a4e9f69bcad604ab421e85c86c...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,25435
8,Deviant for 7 Years,Female/Moldova,376,0,140,June 21,417,0,"23,291",0,...,70,"August 26, 2013",NaN,900,NaN,https://www.deviantart.com/nadia-ch/art/Sherlo...,[full/7dd4af05bd4fca98ce1688c899c444e3e6e12c0d...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,9546
9,Deviant for 10 Years,United Kingdom,17,0,77,NaN,6,0,"18,467",0,...,126,"August 1, 2010",19521.0,3140,NaN,https://www.deviantart.com/1stclassstamps/art/...,[full/df2ff494fd1108fb461550692608c082c4043056...,[https://images-wixmp-ed30a86b8c4ca887773594c2...,Sherlock,46933
